In [10]:
import tkinter as tk
from tkinter import ttk, messagebox
import json
from datetime import datetime

class NodoBST:
    def __init__(self, producto):
        self.producto = producto
        self.izquierda = None
        self.derecha = None

class ArbolBST:
    def __init__(self):
        self.raiz = None
    
    def insertar(self, producto):
        if not self.raiz:
            self.raiz = NodoBST(producto)
        else:
            self._insertar_recursivo(self.raiz, producto)
    
    def _insertar_recursivo(self, nodo, producto):
        if producto['codigo'] < nodo.producto['codigo']:
            if nodo.izquierda is None:
                nodo.izquierda = NodoBST(producto)
            else:
                self._insertar_recursivo(nodo.izquierda, producto)
        else:
            if nodo.derecha is None:
                nodo.derecha = NodoBST(producto)
            else:
                self._insertar_recursivo(nodo.derecha, producto)
    
    def inorder(self):
        productos = []
        def _inorder_recursivo(nodo):
            if nodo:
                _inorder_recursivo(nodo.izquierda)
                productos.append(nodo.producto)
                _inorder_recursivo(nodo.derecha)
        _inorder_recursivo(self.raiz)
        return productos
    
    def preorder(self):
        productos = []
        def _preorder_recursivo(nodo):
            if nodo:
                productos.append(nodo.producto)
                _preorder_recursivo(nodo.izquierda)
                _preorder_recursivo(nodo.derecha)
        _preorder_recursivo(self.raiz)
        return productos
    
    def buscar(self, codigo):
        return self._buscar_recursivo(self.raiz, codigo)
    
    def _buscar_recursivo(self, nodo, codigo):
        if nodo is None or nodo.producto['codigo'] == codigo:
            return nodo
        if codigo < nodo.producto['codigo']:
            return self._buscar_recursivo(nodo.izquierda, codigo)
        return self._buscar_recursivo(nodo.derecha, codigo)

class AlmacenGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistema de Gestión de Almacenes con BST")
        self.root.geometry("1000x600")
        
        # Inicializar árbol BST
        self.arbol = ArbolBST()
        
        # Cargar datos iniciales
        self.cargar_datos()
        
        # Crear pestañas
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(expand=True, fill='both', padx=10, pady=5)
        
        # Pestañas
        self.tab_visualizar = ttk.Frame(self.notebook)
        self.tab_insertar = ttk.Frame(self.notebook)
        self.tab_buscar = ttk.Frame(self.notebook)
        self.tab_ordenamiento = ttk.Frame(self.notebook)
        
        self.notebook.add(self.tab_visualizar, text='Visualizar Inventario')
        self.notebook.add(self.tab_insertar, text='Insertar Producto')
        self.notebook.add(self.tab_buscar, text='Buscar Producto')
        self.notebook.add(self.tab_ordenamiento, text='Ordenamientos BST')
        
        self.setup_visualizar_tab()
        self.setup_insertar_tab()
        self.setup_buscar_tab()
        self.setup_ordenamiento_tab()
        
    def cargar_datos(self):
        try:
            with open('inventario.txt', 'r') as file:
                contenido = file.read()
                self.inventario = []
                for item in contenido.strip().split('}'):
                    if item.strip():
                        if not item.endswith('}'):
                            item += '}'
                        item = item.strip().lstrip('{').strip()
                        if item:
                            try:
                                producto = json.loads('{' + item)
                                # Generar código si no existe
                                if 'codigo' not in producto:
                                    producto['codigo'] = str(len(self.inventario) + 1).zfill(4)
                                self.inventario.append(producto)
                                self.arbol.insertar(producto)
                            except json.JSONDecodeError:
                                continue
        except FileNotFoundError:
            self.inventario = []
            messagebox.showwarning("Advertencia", "No se encontró el archivo inventarios1.txt")
    
    def setup_ordenamiento_tab(self):
        # Frame para botones
        btn_frame = ttk.Frame(self.tab_ordenamiento)
        btn_frame.pack(fill='x', padx=5, pady=5)
        
        ttk.Button(btn_frame, text="Ver Inorder", 
                  command=lambda: self.mostrar_ordenamiento('inorder')).pack(side='left', padx=5)
        ttk.Button(btn_frame, text="Ver Preorder", 
                  command=lambda: self.mostrar_ordenamiento('preorder')).pack(side='left', padx=5)
        
        # Crear Treeview para resultados
        columns = ('Código', 'ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree_ordenamiento = ttk.Treeview(self.tab_ordenamiento, columns=columns, show='headings')
        
        for col in columns:
            self.tree_ordenamiento.heading(col, text=col)
            self.tree_ordenamiento.column(col, width=100)
        
        # Scrollbar
        scrollbar = ttk.Scrollbar(self.tab_ordenamiento, orient='vertical', 
                                command=self.tree_ordenamiento.yview)
        self.tree_ordenamiento.configure(yscrollcommand=scrollbar.set)
        
        self.tree_ordenamiento.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
    
    def mostrar_ordenamiento(self, tipo):
        # Limpiar vista actual
        for item in self.tree_ordenamiento.get_children():
            self.tree_ordenamiento.delete(item)
        
        # Obtener lista ordenada
        if tipo == 'inorder':
            productos = self.arbol.inorder()
        else:  # preorder
            productos = self.arbol.preorder()
        
        # Mostrar productos
        for producto in productos:
            self.tree_ordenamiento.insert('', 'end', values=(
                producto['codigo'],
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))
    
    def setup_visualizar_tab(self):
        columns = ('Código', 'ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree = ttk.Treeview(self.tab_visualizar, columns=columns, show='headings')
        
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)
        
        scrollbar = ttk.Scrollbar(self.tab_visualizar, orient='vertical', command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        self.tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        
        self.actualizar_treeview()
    
    def setup_insertar_tab(self):
        # Añadir campo de código
        ttk.Label(self.tab_insertar, text="Código (4 dígitos):").grid(row=0, column=0, padx=5, pady=5)
        self.codigo_entry = ttk.Entry(self.tab_insertar)
        self.codigo_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="ID:").grid(row=1, column=0, padx=5, pady=5)
        self.id_entry = ttk.Entry(self.tab_insertar)
        self.id_entry.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Nombre:").grid(row=2, column=0, padx=5, pady=5)
        self.nombre_entry = ttk.Entry(self.tab_insertar)
        self.nombre_entry.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Tipo:").grid(row=3, column=0, padx=5, pady=5)
        self.tipo_combo = ttk.Combobox(self.tab_insertar, values=['perecedero', 'no perecedero'])
        self.tipo_combo.grid(row=3, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Cantidad:").grid(row=4, column=0, padx=5, pady=5)
        self.cantidad_entry = ttk.Entry(self.tab_insertar)
        self.cantidad_entry.grid(row=4, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Fecha Vencimiento:").grid(row=5, column=0, padx=5, pady=5)
        self.fecha_entry = ttk.Entry(self.tab_insertar)
        self.fecha_entry.grid(row=5, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_insertar, text="Insertar Producto", 
                  command=self.insertar_producto).grid(row=6, column=0, columnspan=2, pady=20)
    
    def insertar_producto(self):
        try:
            codigo = self.codigo_entry.get()
            if not codigo or len(codigo) != 4 or not codigo.isdigit():
                messagebox.showerror("Error", "El código debe tener exactamente 4 dígitos")
                return
                
            nuevo_producto = {
                'codigo': codigo,
                'id': int(self.id_entry.get()),
                'nombre': self.nombre_entry.get(),
                'tipo': self.tipo_combo.get(),
                'cantidad': int(self.cantidad_entry.get()),
                'fech_venc': self.fecha_entry.get()
            }
            
            if not nuevo_producto['nombre'] or not nuevo_producto['tipo']:
                messagebox.showerror("Error", "Todos los campos son obligatorios")
                return
            
            # Verificar si el código ya existe
            if self.arbol.buscar(codigo):
                messagebox.showerror("Error", "El código ya existe")
                return
            
            self.inventario.append(nuevo_producto)
            self.arbol.insertar(nuevo_producto)
            self.guardar_datos()
            self.actualizar_treeview()
            
            # Limpiar campos
            for entry in [self.codigo_entry, self.id_entry, self.nombre_entry, 
                         self.cantidad_entry, self.fecha_entry]:
                entry.delete(0, 'end')
            self.tipo_combo.set('')
            
            messagebox.showinfo("Éxito", "Producto insertado correctamente")
            
        except ValueError:
            messagebox.showerror("Error", "Por favor, ingrese valores válidos")
    
    def setup_buscar_tab(self):
        # Añadir búsqueda por código
        ttk.Label(self.tab_buscar, text="Buscar por código:").grid(row=0, column=0, padx=5, pady=5)
        self.buscar_codigo_entry = ttk.Entry(self.tab_buscar)
        self.buscar_codigo_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_buscar, text="Buscar por nombre:").grid(row=1, column=0, padx=5, pady=5)
        self.buscar_nombre_entry = ttk.Entry(self.tab_buscar)
        self.buscar_nombre_entry.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_buscar, text="Buscar por tipo:").grid(row=2, column=0, padx=5, pady=5)
        self.buscar_tipo_combo = ttk.Combobox(self.tab_buscar, values=['perecedero', 'no perecedero'])
        self.buscar_tipo_combo.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_buscar, text="Buscar", 
                  command=self.buscar_productos).grid(row=3, column=0, columnspan=2, pady=20)
        
        columns = ('Código', 'ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree_busqueda = ttk.Treeview(self.tab_buscar, columns=columns, show='headings')
        
        for col in columns:
            self.tree_busqueda.heading(col, text=col)
            self.tree_busqueda.column(col, width=100)
        
        self.tree_busqueda.grid(row=4, column=0, columnspan=2, padx=5, pady=5)
    
    def buscar_productos(self):
        for item in self.tree_busqueda.get_children():
            self.tree_busqueda.delete(item)
            
        codigo_buscar = self.buscar_codigo_entry.get()
        nombre_buscar = self.buscar_nombre_entry.get().lower()
        tipo_buscar = self.buscar_tipo_combo.get()
        
        if codigo_buscar:
            # Búsqueda por código usando BST
            nodo = self.arbol.buscar(codigo_buscar)
            if nodo:
                self.tree_busqueda.insert('', 'end', values=(
                    nodo.producto['codigo'],
                    nodo.producto['id'],
                    nodo.producto['nombre'],
                    nodo.producto['tipo'],
                    nodo.producto['cantidad'],
                    nodo.producto['fech_venc']
                ))
        else:
            # Búsqueda por nombre/tipo usando la lista
            resultados = []
            for producto in self.inventario:
                if ((not nombre_buscar or nombre_buscar in producto['nombre'].lower()) and
                    (not tipo_buscar or tipo_buscar == producto['tipo'])):
                    resultados.append(producto)
            
            # Mostrar resultados
            for producto in resultados:
                self.tree_busqueda.insert('', 'end', values=(
                    producto['codigo'],
                    producto['id'],
                    producto['nombre'],
                    producto['tipo'],
                    producto['cantidad'],
                    producto['fech_venc']
                ))
            
            if not resultados:
                messagebox.showinfo("Búsqueda", "No se encontraron productos")

    def actualizar_treeview(self):
        # Limpiar Treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Insertar datos ordenados por código
        productos_ordenados = self.arbol.inorder()
        for producto in productos_ordenados:
            self.tree.insert('', 'end', values=(
                producto['codigo'],
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))

    def guardar_datos(self):
        with open('inventario.txt', 'w') as file:
            for producto in self.inventario:
                file.write(str(producto))

if __name__ == '__main__':
    root = tk.Tk()
    app = AlmacenGUI(root)
    root.mainloop()